# HW 2 Part 2: PCA and Clustering on the Bank Marketing Dataset

The original dataset is available here: https://archive.ics.uci.edu/ml/datasets/Bank+Marketing
We have a modified subset of this dataset to practice data  preprocessing. 

Perform the following tasks on the provided clean bank marketing dataset before encoding and scaling (bank-hw1.csv contains the dataset and bank-names.txt contains the description of the original dataset). Complete all the [LP] questions to receive a "low pass" grade on the homework. The [HP] question specifications combine all parts of the homework. Please refer to Canvas for more details.

Note that if you are unable to complete any of the LP questions satisfactorily, you will receive a grade of "revision required". You can revise and resubmit your work in exchange for a token. Please review the syllabus for more information on specifications grading.

**VERY IMPORTANT**: Include **ALL** the references you used for this assignment, including names of classmates you discuss with. Failure to cite your sources counts as an act of academic dishonesty and will be taken seriously without zero tolerance. You will automatically receive a “fail” grade in the homework and further serious penalties may be imposed.

NOTE: You can look for help on the Internet but refrain from referencing too much. Please cite all your sources in your submission. 
When you submit your assignment, you automatically agree to the following statement. If you do not agree, it is your responsibility to provide the reason.

“*I affirm that I have neither given nor received unauthorized help in completing this homework. I am not aware of others receiving such help. I have cited all the sources in the solution file.*”

[LP 1] Import the provided clean data file, your preprocessed dataset from HW1, or clean the dataset here. Extract the following columns:

1. duration: all entries must be non-negative
2. campaign
3. pdays
4. previous

The set of values in each column and their meaning is provided in the bank-names.txt file. You can use the information to make decisions. Please explain your choices if you are preprocessing the dataset again here OR comment that you are using your preprocessed dataset from HW1. 

Next, remove outliers using [z-score](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.zscore.html). Specifically, you are advised to remove values that have a z-score below -3 or above +3. 

In [ ]:
# Your code for LP1 goes here
# Feel free to add cells below this for LP1

[LP 2] 
1. Perform Principal Component Analysis (PCA) on the filtered dataset and extract the top three principal components. Perform the necessary preprocessing steps for the same.
2. Plot the top two principal components in a graph using appropriate labels on the x and y axes.


In [ ]:
# Your code for LP2 goes here
# Feel free to add cells below this for LP2

[HP 1]
Did we lose information by picking only the top three principal components? How much information from the four columns in the filtered components are captured in the the two top principal components? Explain your answer briefly.

In [ ]:
# Your code for HP1 goes here
# Feel free to add cells below this for HP1

<your answer for HP 1 goes here>

[LP 3] Perform K-means clustering on the filtered dataset (before PCA) with k = 3 and plot the results using the first two principal components (after PCA). You may look at the class example for reference.

In [ ]:
# Your code for LP3 goes here
# Feel free to add cells below this for LP3

[HP 2] Use the elbow method to select the best value of k for this dataset and plot your results. What value of k is the appropriate number of clusters for this problem?

In [ ]:
# Your code for HP2 goes here
# Feel free to add cells below this for HP2

< Your answer for HP2 goes here >

# References
Include ALL your references here. 

# What to turn in:
1. The ipynb solution file, which includes the references
2. If you use Colab or GitHub for version control, please share a link to your notebook or GitHub repository